In [1]:
%load_ext google.cloud.bigquery

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information form over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 13 years ago. So if today is 2022.02.15 then the weather we want to forecast is for the date 2009.02.16. You are suppsed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to used BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is, to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck at the first part, you can use the replacement data provided in the second part

In [ ]:
%%bigquery df
SELECT
*,
FROM `bigquery-public-data.samples.gsod`

In [3]:
df.to_csv("bq_data.csv", index=False)

## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [2]:
%%bigquery df
SELECT
*, cast(concat(year, "-", month, "-", day) as date) as date
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2005 AND 2009 AND station_number BETWEEN 725300 AND 726300


Downloading: 100%|████████████████████████████████████████████████████████| 377784/377784 [00:13<00:00, 28082.82rows/s]


### 2. Task 
From here want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009. 

In [4]:
df = df[df['station_number'].between(725300, 725330)]
len(df)

18129

Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate. 

In [5]:
#Now that we have a date column, the year, month and day are redundant columns and can be dropped. 
#We can also drop wban_number, as it doesn't provide any useful information, not already covered in the station_number
df=df.drop(['wban_number', 'year', 'month', 'day'], axis=1)

In [6]:
#num mean sample columns can be used as columns providing weight, however in the scope of this task I will prefer 
#a more naive approach, and hence drop these columns, as they are not directly decisive for the probability of 
#snow.
df=df.drop(['num_mean_temp_samples', 'num_mean_dew_point_samples', 'num_mean_sealevel_pressure_samples', 'num_mean_station_pressure_samples', 'num_mean_station_pressure_samples', 'num_mean_visibility_samples', 'num_mean_wind_speed_samples'], axis=1)

In [7]:
print(df.columns)

Index(['station_number', 'mean_temp', 'mean_dew_point',
       'mean_sealevel_pressure', 'mean_station_pressure', 'mean_visibility',
       'mean_wind_speed', 'max_sustained_wind_speed', 'max_gust_wind_speed',
       'max_temperature', 'max_temperature_explicit', 'min_temperature',
       'min_temperature_explicit', 'total_precipitation', 'snow_depth', 'fog',
       'rain', 'snow', 'hail', 'thunder', 'tornado', 'date'],
      dtype='object')


In [8]:
df=df.drop(['max_temperature_explicit', 'min_temperature_explicit', 'max_sustained_wind_speed', 'max_gust_wind_speed'], axis=1)

In [9]:
df

,station_number,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_station_pressure,mean_visibility,mean_wind_speed,max_temperature,min_temperature,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado,date
103,725326,17.400000,8.100000,NaN,NaN,9.9,8.6,10.400000,NaN,0.00,NaN,False,False,False,False,False,False,2005-01-28
113,725326,35.000000,31.900000,NaN,NaN,3.1,1.4,33.799999,NaN,0.00,NaN,False,False,False,False,False,False,2005-12-24
187,725326,69.400002,51.599998,NaN,NaN,9.6,9.7,55.400002,NaN,0.00,NaN,False,False,False,False,False,False,2005-05-10
228,725326,54.200001,35.700001,NaN,NaN,9.5,7.4,35.599998,NaN,0.00,NaN,False,False,False,False,False,False,2005-03-29
366,725326,66.699997,56.500000,NaN,NaN,9.7,2.6,53.599998,NaN,0.00,NaN,False,False,False,False,False,False,2006-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377749,725300,75.000000,65.699997,1019.700012,995.900024,9.9,5.4,64.000000,NaN,0.03,NaN,False,False,False,False,False,False,2005-07-23
377751,725330,73.000000,61.500000,1012.099976,983.400024,9.9,10.6,63.000000,NaN,0.06,NaN,False,False,False,False,False,False,2005-06-06
377753,725300,37.900002,30.900000,1017.299988,992.299988,5.8,8.8,22.100000,NaN,0.17,NaN,True,True,True,True,True,True,2005-03-25
377754,725320,67.500000,63.299999,1016.099976,992.099976,7.9,3.6,62.599998,NaN,0.45,NaN,True,True,True,True,True,True,2005-08-14


In [10]:
len(df)

18129

In [11]:
#from a first glance seems like the min_temperature contains a lot of nan values, for which we do a check to 
#make sure of it 
count = df["min_temperature"].isna().sum()

print(count)

18129


In [12]:
#min_temperature column should also be dropped as the latter only contains NaN values 
df=df.drop(['min_temperature'], axis=1)

In [13]:
df

,station_number,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_station_pressure,mean_visibility,mean_wind_speed,max_temperature,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado,date
103,725326,17.400000,8.100000,NaN,NaN,9.9,8.6,10.400000,0.00,NaN,False,False,False,False,False,False,2005-01-28
113,725326,35.000000,31.900000,NaN,NaN,3.1,1.4,33.799999,0.00,NaN,False,False,False,False,False,False,2005-12-24
187,725326,69.400002,51.599998,NaN,NaN,9.6,9.7,55.400002,0.00,NaN,False,False,False,False,False,False,2005-05-10
228,725326,54.200001,35.700001,NaN,NaN,9.5,7.4,35.599998,0.00,NaN,False,False,False,False,False,False,2005-03-29
366,725326,66.699997,56.500000,NaN,NaN,9.7,2.6,53.599998,0.00,NaN,False,False,False,False,False,False,2006-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377749,725300,75.000000,65.699997,1019.700012,995.900024,9.9,5.4,64.000000,0.03,NaN,False,False,False,False,False,False,2005-07-23
377751,725330,73.000000,61.500000,1012.099976,983.400024,9.9,10.6,63.000000,0.06,NaN,False,False,False,False,False,False,2005-06-06
377753,725300,37.900002,30.900000,1017.299988,992.299988,5.8,8.8,22.100000,0.17,NaN,True,True,True,True,True,True,2005-03-25
377754,725320,67.500000,63.299999,1016.099976,992.099976,7.9,3.6,62.599998,0.45,NaN,True,True,True,True,True,True,2005-08-14


In [14]:
#from a first glance seems like the snow_depth contains a lot of nan values, for which we do a check to 
#make sure of it 
count = df["snow_depth"].isna().sum()
print(count)

17564


In [15]:
#Checking wether the missing values are because the Nans are when it did not snow. In tis case we can replace 
#missing  values by 0s (it didn\t snow hence the depth of snow is 0)
len(df[(df['snow_depth'].isna()) & (df['snow']==True)])
len(df[(df['snow']==True)])

2142

In [16]:
#It turned out that 96%+ of the values are Nan in the snow_depth column, and while it could be a relevant data field
#and perhaps some sort of interpolation would have been a smart idea if more values were available, interpolating in
#such a scenario may create large unjustified bias based on assumptions, and hence it is preferable to drop the 
#column (if no additional data is available) 
df=df.drop(['snow_depth'], axis=1)
df

,station_number,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_station_pressure,mean_visibility,mean_wind_speed,max_temperature,total_precipitation,fog,rain,snow,hail,thunder,tornado,date
103,725326,17.400000,8.100000,NaN,NaN,9.9,8.6,10.400000,0.00,False,False,False,False,False,False,2005-01-28
113,725326,35.000000,31.900000,NaN,NaN,3.1,1.4,33.799999,0.00,False,False,False,False,False,False,2005-12-24
187,725326,69.400002,51.599998,NaN,NaN,9.6,9.7,55.400002,0.00,False,False,False,False,False,False,2005-05-10
228,725326,54.200001,35.700001,NaN,NaN,9.5,7.4,35.599998,0.00,False,False,False,False,False,False,2005-03-29
366,725326,66.699997,56.500000,NaN,NaN,9.7,2.6,53.599998,0.00,False,False,False,False,False,False,2006-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377749,725300,75.000000,65.699997,1019.700012,995.900024,9.9,5.4,64.000000,0.03,False,False,False,False,False,False,2005-07-23
377751,725330,73.000000,61.500000,1012.099976,983.400024,9.9,10.6,63.000000,0.06,False,False,False,False,False,False,2005-06-06
377753,725300,37.900002,30.900000,1017.299988,992.299988,5.8,8.8,22.100000,0.17,True,True,True,True,True,True,2005-03-25
377754,725320,67.500000,63.299999,1016.099976,992.099976,7.9,3.6,62.599998,0.45,True,True,True,True,True,True,2005-08-14


In [17]:
#mean_sealevel_pressure and mean_station_pressure interpolate values
df['mean_sealevel_pressure'].fillna((df['mean_sealevel_pressure'].mean()), inplace=True)
df['mean_station_pressure'].fillna((df['mean_station_pressure'].mean()), inplace=True)
df

,station_number,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_station_pressure,mean_visibility,mean_wind_speed,max_temperature,total_precipitation,fog,rain,snow,hail,thunder,tornado,date
103,725326,17.400000,8.100000,1016.937585,990.766767,9.9,8.6,10.400000,0.00,False,False,False,False,False,False,2005-01-28
113,725326,35.000000,31.900000,1016.937585,990.766767,3.1,1.4,33.799999,0.00,False,False,False,False,False,False,2005-12-24
187,725326,69.400002,51.599998,1016.937585,990.766767,9.6,9.7,55.400002,0.00,False,False,False,False,False,False,2005-05-10
228,725326,54.200001,35.700001,1016.937585,990.766767,9.5,7.4,35.599998,0.00,False,False,False,False,False,False,2005-03-29
366,725326,66.699997,56.500000,1016.937585,990.766767,9.7,2.6,53.599998,0.00,False,False,False,False,False,False,2006-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377749,725300,75.000000,65.699997,1019.700012,995.900024,9.9,5.4,64.000000,0.03,False,False,False,False,False,False,2005-07-23
377751,725330,73.000000,61.500000,1012.099976,983.400024,9.9,10.6,63.000000,0.06,False,False,False,False,False,False,2005-06-06
377753,725300,37.900002,30.900000,1017.299988,992.299988,5.8,8.8,22.100000,0.17,True,True,True,True,True,True,2005-03-25
377754,725320,67.500000,63.299999,1016.099976,992.099976,7.9,3.6,62.599998,0.45,True,True,True,True,True,True,2005-08-14


In [18]:
#Other Nan values with small quantities are contained in the following 5 columns, we will interpolate the average 
#value for those columns, given the number of missing values is 2-5 in 4 of the columns, and only in 1 of them there 
#are 191 missing values 
df['mean_dew_point'].fillna((df['mean_dew_point'].mean()), inplace=True)
df['mean_visibility'].fillna((df['mean_visibility'].mean()), inplace=True)
df['mean_wind_speed'].fillna((df['mean_wind_speed'].mean()), inplace=True)
df['max_temperature'].fillna((df['max_temperature'].mean()), inplace=True)
df['total_precipitation'].fillna((df['total_precipitation'].mean()), inplace=True)

In [19]:
#how many stations are we considering?
df['station_number'].nunique()

10

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for is the following, and hence should constitute your test set.

In [20]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X=df[['station_number', 'mean_temp', 'mean_dew_point', 'mean_sealevel_pressure', 'mean_station_pressure', 'mean_visibility', 'mean_wind_speed', 'max_temperature', 'total_precipitation', 'fog', 'rain', 'hail', 'thunder', 'tornado']]  # Features
y=df['snow']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [21]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]

'2009-03-04'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [22]:
#normalize the datasets
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
X_train_n = scaler.fit_transform(X_train)
X_test_n = scaler.fit_transform(X_test)
y_train_n = scaler.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_n = scaler.fit_transform(y_test.to_numpy().reshape(-1, 1))

In [23]:
print(X_train_n)
print("--------------")
print(X_test_n)
print("--------------")
print(y_train_n)
print("--------------")
print(y_test_n)

[[0.46666667 0.88724587 0.92884995 ... 1.         1.         1.        ]
 [0.46666667 0.30776341 0.31968812 ... 0.         0.         0.        ]
 [0.53333333 0.90942696 0.94054582 ... 0.         0.         0.        ]
 ...
 [1.         0.59242146 0.52339183 ... 0.         0.         0.        ]
 [1.         0.59057302 0.58966862 ... 0.         0.         0.        ]
 [0.86666667 0.79020336 0.83625731 ... 0.         0.         0.        ]]
--------------
[[0.46666667 0.80765458 0.7138493  ... 0.         0.         0.        ]
 [0.16666667 0.43572129 0.45010185 ... 0.         0.         0.        ]
 [0.86666667 0.83709521 0.90733199 ... 0.         0.         0.        ]
 ...
 [0.46666667 0.49067712 0.39205703 ... 0.         0.         0.        ]
 [0.56666667 0.31501473 0.32077393 ... 0.         0.         0.        ]
 [0.66666667 0.54170755 0.56822812 ... 0.         0.         0.        ]]
--------------
[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
--------------
[[0.]
 [0.]
 [0.]
 ...
 [

In [ ]:
#Import Random Forest Model for predictions (since it is pretty robust and well used for similar binary classifications)
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_n,y_train_n)

y_pred=clf.predict(X_test_n)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_n, y_pred))

In [ ]:
#Import svm model for predictions since random forest clearly overfitted the training set
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train_n, y_train_n.ravel())

#Predict the response for test dataset
y_pred = clf.predict(X_test_n)

In [26]:
# predict using k-fold cross-validation for overcoming overfitting
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
# evaluate model
scores = cross_val_score(model, X_test_n, y_test_n, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 1.000 (0.000)


In [ ]:
#all three models yielded in a 100% accuracy, and cross validation did not change anything, which means probably 
#the issue is not overfitting. One idea could be that perhaps there is a column or columns with values exactly 
#duplicating those in the column we are trying to predict, that being snow. Below are  checks for identifying 
#such columns


In [27]:
df.loc[((df['snow'] == False) & (df['fog']==False)) | ((df['snow'] == True) & (df['fog']==True))]

,station_number,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_station_pressure,mean_visibility,mean_wind_speed,max_temperature,total_precipitation,fog,rain,snow,hail,thunder,tornado,date
103,725326,17.400000,8.100000,1016.937585,990.766767,9.9,8.6,10.400000,0.00,False,False,False,False,False,False,2005-01-28
113,725326,35.000000,31.900000,1016.937585,990.766767,3.1,1.4,33.799999,0.00,False,False,False,False,False,False,2005-12-24
187,725326,69.400002,51.599998,1016.937585,990.766767,9.6,9.7,55.400002,0.00,False,False,False,False,False,False,2005-05-10
228,725326,54.200001,35.700001,1016.937585,990.766767,9.5,7.4,35.599998,0.00,False,False,False,False,False,False,2005-03-29
366,725326,66.699997,56.500000,1016.937585,990.766767,9.7,2.6,53.599998,0.00,False,False,False,False,False,False,2006-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377749,725300,75.000000,65.699997,1019.700012,995.900024,9.9,5.4,64.000000,0.03,False,False,False,False,False,False,2005-07-23
377751,725330,73.000000,61.500000,1012.099976,983.400024,9.9,10.6,63.000000,0.06,False,False,False,False,False,False,2005-06-06
377753,725300,37.900002,30.900000,1017.299988,992.299988,5.8,8.8,22.100000,0.17,True,True,True,True,True,True,2005-03-25
377754,725320,67.500000,63.299999,1016.099976,992.099976,7.9,3.6,62.599998,0.45,True,True,True,True,True,True,2005-08-14


In [ ]:
#So apparently the fog column precisely defined the feasibility of the snow on a given date. The investigations 
#for the narrow scope of this assignment are to be terminated here. However in real life scenarios it would just mean
#that we either have ill data, or the data at hand is insufficient, so the next logical steps would be communicaing
#over 